## Análise de Sentimentos – utilizando dados do Twitter 

### Importa todos os pacotes de Processamento de Linguagem Natural

In [1]:
#import nltk
#nltk.download('all')
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

**Ler arquivo de dados e conta a quantidade de linhas**

In [2]:
# Dados coletados e Classificados pelo Data Scientist - Rodrigo Santana Ferreira 
dataset = pd.read_csv('Tweets_Mg.csv',encoding='utf-8')

In [3]:
# número de tuplas
print("Número de linhas: ",len(dataset))

Número de linhas:  8199


In [4]:
# número de atributos
print("Número de atributos: ",len(dataset.columns))

Número de atributos:  26


In [5]:
# atributos
dataset.columns

Index(['Unnamed: 0', 'Created At', 'Text', 'Geo Coordinates.latitude',
       'Geo Coordinates.longitude', 'User Location', 'Username',
       'User Screen Name', 'Retweet Count', 'Classificacao', 'Observação',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [6]:
# tweets Já classificados - Positivos, Negativos e Neutros
dataset.head(8)

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Sat Jan 07 13:47:55 +0000 2017,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",NaN,NaN,Um Outro Mundo É Possível,Maristela Guimarães,florpimenta,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Wed Jan 04 23:00:53 +0000 2017,"""..E 25% dos mineiros dizem não torcer para ti...",NaN,NaN,NaN,Adriano,almanakfc,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Sun Jan 08 18:34:22 +0000 2017,"""A gigantesca barba do mal"" em destaque no cad...",NaN,NaN,"Belo Horizonte, Brasil",Editora Nemo,editoranemo,2,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Remover emoji

In [ ]:
#!pip install emoji

In [7]:
# remover emoji
import emoji

def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI] + ['�', '@']
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

#text = dataset.text
#text_tratado = give_emoji_free_text(text)
dataset.Text = dataset.Text.apply(lambda w: give_emoji_free_text(w))
dataset.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,Catedral de Santo Antônio - Governador Valadar...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"Governador Valadares, Minas Gerais https://t.c...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"Governador Valadares, Minas Gerais https://t.c...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,PSOL vai questionar aumento de vereadores e pr...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# eliminando alguns atributos do dataset

In [8]:
# eliminando alguns atributos do dataset
atributos = ['Text', 'Classificacao', 'Created At', 'Geo Coordinates.latitude', 'Geo Coordinates.longitude',
             'User Location', 'User Screen Name',  ]
atributos = ['Text', 'Classificacao', 'Created At'  ]
dataset2 = dataset.loc[:,atributos]
dataset = dataset2
dataset.head(8)

,Text,Classificacao,Created At
0,Catedral de Santo Antônio - Governador Valadar...,Neutro,Sun Jan 08 01:22:05 +0000 2017
1,"Governador Valadares, Minas Gerais https://t.c...",Neutro,Sun Jan 08 01:49:01 +0000 2017
2,"Governador Valadares, Minas Gerais https://t.c...",Neutro,Sun Jan 08 01:01:46 +0000 2017
3,https://t.co/BnDsO34qK0,Neutro,Wed Jan 04 21:43:51 +0000 2017
4,PSOL vai questionar aumento de vereadores e pr...,Negativo,Mon Jan 09 15:08:21 +0000 2017
5,""" bom é bandido morto"" Deputado Cabo Júlio é c...",Neutro,Sat Jan 07 13:47:55 +0000 2017
6,"""..E 25% dos mineiros dizem não torcer para ti...",Neutro,Wed Jan 04 23:00:53 +0000 2017
7,"""A gigantesca barba do mal"" em destaque no cad...",Neutro,Sun Jan 08 18:34:22 +0000 2017


In [9]:
dataset.tail()

,Text,Classificacao,Created At
8194,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,Thu Feb 09 11:48:07 +0000 2017
8195,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,Thu Feb 09 12:10:19 +0000 2017
8196,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,Thu Feb 09 12:04:17 +0000 2017
8197,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,Thu Feb 09 12:10:04 +0000 2017
8198,Trio suspeito de roubo de cargas é preso em Sa...,Positivo,Tue Feb 07 15:01:30 +0000 2017


**Conta a quantidade de linhas de tweets neutros, positivos e negativos**

In [10]:
dataset.Classificacao.value_counts()

Positivo    3300
Neutro      2453
Negativo    2446
Name: Classificacao, dtype: int64

In [11]:
len(dataset[dataset.Classificacao=='Neutro'])

2453

In [12]:
len(dataset[dataset.Classificacao=='Positivo'])

3300

In [13]:
len(dataset[dataset.Classificacao=='Negativo'])

2446

## Pre-Processamento 

** Separando tweets e suas classes
**

In [14]:
tweets = dataset['Text'].values
classes = dataset['Classificacao'].values

In [15]:
print(tweets[:10])

['Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ'
 'Governador Valadares, Minas Gerais https://t.co/B3ThIDJCSf'
 'Governador Valadares, Minas Gerais https://t.co/dPkgzVR2Qw'
 'https://t.co/BnDsO34qK0'
 'PSOL vai questionar aumento de vereadores e prefeito de BH na Justiça - Politica - Estado de Minas https://t.co/DMg7BGsek5'
 '" bom é bandido morto" Deputado Cabo Júlio é condenado e fica inelegível por 10 anos - Politica - Estado de Minas https://t.co/3GfAqvrFHS'
 '"..E 25% dos mineiros dizem não torcer para time nenhum,mesmo dentro de um estado com Atlético-MG e Cruzeiro. Pq?.." https://t.co/fN5evlLQsR'
 '"A gigantesca barba do mal" em destaque no caderno Cultura do Estado de Minas. https://t.co/PxNurIkTlw'
 '"BB e governo de Minas travam disputa sobre depósitos judiciais" https://t.co/CnMu2A2Qo5'
 '"com vcs bh fica pequena!" Belo Horizonte (pron. [bɛloɾiˈzõntʃi][10]) é a capital do estado de MG, com uma área de aproximadamente 331 km²']


In [16]:
classes[:10]

array(['Neutro', 'Neutro', 'Neutro', 'Neutro', 'Negativo', 'Neutro',
       'Neutro', 'Neutro', 'Negativo', 'Neutro'], dtype=object)

**Gerando o modelo**

In [17]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets)
freq_tweets.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:
print(vectorizer.get_feature_names()[:-100])

['00', '000', '001', '00qmalz7fo', '01', '01a3ewki4z', '01ecfljipq', '028', '02jbfmpvn7', '03', '04', '040', '04124011533', '04162571719', '049r6iagih', '04lx7izoh8', '04phytiopz', '04wzmbaarf', '05', '050', '06', '07', '08', '08flxax56y', '08m5ha73xy', '08s4s5hxfv', '09', '0a0dacmlrs', '0abns6zvef', '0aninsardr', '0asli14vlk', '0bnzbajuok', '0btfbfr2bc', '0by1m090fv', '0c1kbavzrj', '0cwj32qq88', '0d7j0dllwg', '0d7j4ddf8g', '0dldpz9wgz', '0du0pujm70', '0dyufu0cy9', '0enjyioxcq', '0exmlnmkr8', '0f1ppmlnug', '0fhvl1bzrq', '0fiuadaja9', '0fjuqqjpez', '0guawnr7rj', '0gy1x2bgdj', '0hddidctgp', '0hqkwrusth', '0iehkx193x', '0ingjcmtzb', '0je63rjibu', '0jfjlhzawp', '0jh7arbsnk', '0jtpm5lqcb', '0kdsvch67v', '0l6jzdx5tm', '0l7o2zyyul', '0ldpnxsyzv', '0lmwsbalpt', '0m2x7mfdij', '0mdjvpnnpv', '0mmb63ijul', '0mzc3rhw4t', '0nbvvbyveg', '0nrhwylcz2', '0ort7r8gmo', '0pkpiuh91b', '0plmmyocsi', '0qfkqrpyxr', '0qkczrvj26', '0qmjaflb0n', '0qomj13uzq', '0qqah2obah', '0qr6qocx43', '0qtgyd2pfs', '0qzen7zkjh'

In [21]:
freq_tweets.shape

(8199, 12296)

(8199, 12296) - Novo
(8199, 13029) - Antigo (com emoji)

In [22]:
modelo = MultinomialNB() # Bayes
modelo.fit(freq_tweets,classes)

MultinomialNB()

** Testando o modelo com algumas instâncias simples**

In [25]:
testes = ['Esse governo está no início, vamos ver o que vai dar',
          'Estou muito feliz com o governo de Minas esse ano',
          'O estado de Minas Gerais decretou calamidade financeira!!!',
          'A segurança desse país está deixando a desejar',
          'O governador de Minas é do PT']

In [26]:
freq_testes = vectorizer.transform(testes)

In [27]:
freq_testes.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [31]:
print(vectorizer.get_feature_names()[-100:])

['zohllpax9w', 'zona', 'zonas', 'zoqrb2ul3y', 'zorra', 'zp4i49i9gg', 'zp7nu94pip', 'zphsc3gzqd', 'zpjdimiwua', 'zpmuc7ckhw', 'zqce5mrgkf', 'zqgejpu91n', 'zqi', 'zqlb256xcs', 'zqwbburt9m', 'zqx6ikojpq', 'zqyd3mf8ot', 'zr9n6gl0kw', 'zraef33t5x', 'zrdk8b95dp', 'zrorb7wqvj', 'zrvoddrqmx', 'zs1etxeaan', 'zsd6wjez81', 'zsgbum3o4e', 'zsjjvgzyos', 'zsoqtm4rck', 'zsvicxvk4g', 'zsyjqpjzrx', 'zsz6mpraxn', 'zt4qhla10z', 'zte5xq0dyd', 'zti8teclxw', 'ztjpvmmx6e', 'ztlv54pcjm', 'ztpoffmzlz', 'ztrz8mgrrp', 'ztvq2ym4aj', 'ztww3u9epm', 'zuapghxzpa', 'zuar', 'zuejudw7xs', 'zulia', 'zurdos', 'zuyjfmh6jo', 'zvhwo4czah', 'zvr2targsb', 'zvrzvxdphk', 'zvvukpnmai', 'zvw2v3jkwj', 'zw4zuzlx8k', 'zwi7kis4zy', 'zwlompgs4u', 'zx55pdbxb2', 'zx5ipheoce', 'zxcaxoshcx', 'zxdfyqiu7q', 'zxkq7uyaum', 'zxnvxcjw7m', 'zxp0aku6vr', 'zxrmm78yyj', 'zxu0kyji3q', 'zygmunt', 'zyp224zjcc', 'zysr7hnu1e', 'zytbmnch2h', 'zze0wfnyil', 'zze9oxap3c', 'zzf7nfb4et', 'zzm1tcutjz', 'zznv9jkmop', 'zzpbswmcyx', 'zzqawnrxqc', 'zzqrgwnlvl', 'zé'

In [32]:
# Fazendo a classificação com o modelo treinado.
modelo.predict(freq_testes)

array(['Neutro', 'Neutro', 'Negativo', 'Negativo', 'Neutro'], dtype='<U8')

In [33]:
#df = pd.DataFrame()
#df['tweet'] = testes
#df['classificacao'] = modelo.predict(freq_testes)

tweets = testes
classificacao = modelo.predict(freq_testes)

print ("Novas Instancias Classificadas...")
for i in range(len(testes)):
    print (classificacao[i], " -> ", tweets[i] )

Novas Instancias Classificadas...
Neutro  ->  Esse governo está no início, vamos ver o que vai dar
Neutro  ->  Estou muito feliz com o governo de Minas esse ano
Negativo  ->  O estado de Minas Gerais decretou calamidade financeira!!!
Negativo  ->  A segurança desse país está deixando a desejar
Neutro  ->  O governador de Minas é do PT


** Avaliando o modelo **

In [34]:
# Fazendo o cross validation do modelo
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [35]:
freq_tweets.count_nonzero()

124163

In [36]:
resultados[:10]

array(['Neutro', 'Neutro', 'Neutro', 'Positivo', 'Neutro', 'Neutro',
       'Neutro', 'Neutro', 'Negativo', 'Neutro'], dtype='<U8')

In [37]:
# Medindo a acurácia média do modelo
metrics.accuracy_score(classes,resultados)

0.8832784485912917

0.8832784485912917 - Sem emoji
0.8831564824978656 - com emoji

### Matriz de Confusão

In [38]:
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['REAL'], colnames=['PREDITO'], margins=True))

PREDITO   Negativo  Neutro  Positivo   All
REAL                                      
Negativo      2282     157         7  2446
Neutro         248    2068       137  2453
Positivo        46     362      2892  3300
All           2576    2587      3036  8199


PREDITO   Negativo  Neutro  Positivo   All
REAL                                      
Negativo      2275     162         9  2446
Neutro         240    2067       146  2453
Positivo        45     356      2899  3300
All           2560    2585      3054  8199

In [39]:
# Medidas de validação do modelo
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

              precision    recall  f1-score   support

    Positivo       0.95      0.88      0.91      3300
    Negativo       0.89      0.93      0.91      2446
      Neutro       0.80      0.84      0.82      2453

    accuracy                           0.88      8199
   macro avg       0.88      0.88      0.88      8199
weighted avg       0.89      0.88      0.88      8199



/Users/aulas_uepb/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass labels=['Positivo', 'Negativo', 'Neutro'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
